In [76]:
%%capture
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph
%pip install --upgrade --quiet  langchain-openai
%pip install -qU langchain-core
%pip install -qU langchain-chroma

In [77]:
import os
from openai import AzureOpenAI

llm = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-02-01"
)


In [78]:
log_file_path = "C:/Users/kr4193/Desktop/Log_error_reporter/Prep_work/clean_Geiger_for_LLMs.log"
pytest_path = "C:/Users/kr4193/Downloads/evtstuff/atf/tests/print/clean_cutter.py"
# Read the log content from the file

file = open(pytest_path, "r")
content = file.read()
def read_log_file(file_path):
    try:
        with open(file_path, 'r') as file:
            return file.read()
    except FileNotFoundError:
        print(f"Error: The file at {file_path} was not found.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
documents = read_log_file(log_file_path)

In [79]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings( 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-02-01",
  model="gpt-4o")

In [80]:
from langchain_chroma import Chroma

vector_store = Chroma(embedding_function=embeddings)

In [81]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)

docs = loader.load()

assert len(docs) == 1
print(f"Total characters: {len(docs[0].page_content)}")

Total characters: 43130


In [82]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

Split blog post into 66 sub-documents.


In [83]:
document_ids = vector_store.add_documents(documents=all_splits)

print(document_ids[:3])

APIConnectionError: Connection error.